In [1]:
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
# from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from itertools import product
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import pickle

In [2]:
df = pd.read_csv('dataset.csv', index_col = "DATE")
df

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,AWND,FMTM,PGTM,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,city,state,region,th_power,power
DATE,,,,,,,,,,,,,,,,,,,,
2010-01-01,USW00014942,"OMAHA EPPLEY AIRFIELD, NE US",41.31186,-95.90186,298.8,3.800000,1311.000000,1159.000000,11.419891,-11.100000,-23.30000,350.000000,330.000000,8.000000,10.300000,Des Moines,IA,Midwest Region,105682.164827,0
2010-01-02,USW00014942,"OMAHA EPPLEY AIRFIELD, NE US",41.31186,-95.90186,298.8,1.700000,222.000000,1950.000000,11.419891,-16.700000,-25.60000,10.000000,30.000000,5.400000,7.200000,Des Moines,IA,Midwest Region,0.000000,0
2010-01-03,USW00014942,"OMAHA EPPLEY AIRFIELD, NE US",41.31186,-95.90186,298.8,2.400000,1547.000000,1558.000000,11.419891,-13.300000,-20.60000,10.000000,20.000000,5.400000,5.800000,Des Moines,IA,Midwest Region,0.000000,0
2010-01-04,USW00014942,"OMAHA EPPLEY AIRFIELD, NE US",41.31186,-95.90186,298.8,2.700000,1606.000000,1605.000000,11.419891,-15.600000,-28.90000,10.000000,10.000000,4.000000,4.500000,Des Moines,IA,Midwest Region,0.000000,0
2010-01-05,USW00014942,"OMAHA EPPLEY AIRFIELD, NE US",41.31186,-95.90186,298.8,1.300000,2219.000000,2221.000000,11.419891,-13.300000,-26.70000,150.000000,140.000000,3.600000,4.000000,Des Moines,IA,Midwest Region,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-29,USW00013881,"CHARLOTTE DOUGLAS AIRPORT, NC US",35.22254,-80.95433,222.6,2.300000,1836.559043,1422.573668,18.900000,22.200000,15.60000,150.000000,150.000000,4.000000,5.800000,Charleston,SC,Coastal Areas,0.000000,0
2023-04-30,USW00013881,"CHARLOTTE DOUGLAS AIRPORT, NC US",35.22254,-80.95433,222.6,3.800000,1836.559043,1422.573668,18.500000,22.200000,10.00000,300.000000,300.000000,12.500000,17.900000,Charleston,SC,Coastal Areas,105682.164827,0
2023-05-01,USW00013881,"CHARLOTTE DOUGLAS AIRPORT, NC US",35.22254,-80.95433,222.6,5.200000,1836.559043,1422.573668,14.100000,20.000000,8.30000,220.000000,260.000000,11.200000,14.800000,Charleston,SC,Coastal Areas,270807.658405,0


In [3]:
def adfuller_test(windSpeed):
    result=adfuller(windSpeed)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

In [4]:
for station in df.STATION.unique():    
    temp_df = df[df['STATION'].isin({station})]
    model=ARIMA(temp_df['th_power'],order=(1,0,2))
    model_fit=model.fit()
    model_fit.summary()
    Pkl_Filename = "Power models/ARIMA/"+station+".pkl"  
    with open(Pkl_Filename, 'wb') as file:  
        pickle.dump(model, file)
    

In [5]:
# temp_df = data[data['STATION'].isin({'USW00014942'})]
# plt.figure(figsize=[15, 7.5]); # Set dimensions for figure
# plt.plot(temp_df.index, temp_df['AWND'])
# plt.title('Quarterly EPS for Johnson & Johnson')
# plt.ylabel('EPS per share ($)')
# plt.xlabel('Date')
# plt.xticks(rotation=90)
# plt.grid(True)
# plt.show()

In [6]:
# plot_pacf(temp_df['AWND'], lags=50);
# plot_acf(temp_df['AWND'], lags=50);

In [7]:

# def adfuller_test(windSpeed):
#     result=adfuller(windSpeed)
#     labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
#     for value,label in zip(result,labels):
#         print(label+' : '+str(value) )
#     if result[1] <= 0.05:
#         print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
#     else:
#         print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

In [8]:
# adfuller_test(temp_df['AWND'])

In [9]:
# from pandas.plotting import autocorrelation_plot
# autocorrelation_plot(temp_df['AWND'])
# plt.show()

In [10]:
# fig = plt.figure(figsize=(12,8))
# ax1 = fig.add_subplot(211)
# fig = plot_acf(temp_df['AWND'],lags=40,ax=ax1)
# ax2 = fig.add_subplot(212)
# fig = plot_pacf(temp_df['AWND'],lags=40,ax=ax2)

In [11]:
# from statsmodels.tsa.arima.model import ARIMA
# model=ARIMA(temp_df['AWND'],order=(1,0,2))
# model_fit=model.fit()
# model_fit.summary()
# # type(model_fit)

In [12]:
# temp_df['forecast']=model_fit.predict(start=90,end=103,dynamic=True)
# temp_df[['AWND','forecast']].plot(figsize=(12,8))